# Provisioning OpenRouter API keys

This notebook is for instructors to provision OpenRouter API keys for workshop participants. It is not intended for use by participants.

However participants are welcome to see the code and use it as a reference whenever they need to bulk provision API keys for their own projects.

## OpenRouter API provisioning

OpenRouter provides a simple API to create and manage API keys. 

The core idea is to use a special API key that has the `admin` role, which allows you to create new API keys for other users. This key does not make regular API calls but is used to manage API keys.

This type of key is extremely powerful and should be kept secret. It is not intended for use in production applications or by end users.

If this type of key is compromised, it can be used to create new API keys for any user, which can then be used to make API calls and drain your account's balance to zero.